In [1]:
#Initialize imports
from distutils import core
import os
import requests
import json 
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import tweepy
import time
load_dotenv()

True

In [2]:
#bring in API keys
twitter_api_key = os.getenv("TWITTER_API_KEY")
twitter_secret_key = os.getenv("TWITTER_SECRET_KEY")
twitter_access_token = os.getenv("TWITTER_ACCESS_TOKEN")
twitter_secret_token = os.getenv("TWITTER_SECRET_TOKEN")
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [3]:
#baseurl is used for alpaca and userid is used for twitter
base_url = 'https://paper-api.alpaca.markets'
user_id ='News_Of_Alpha'
url = 'https://twitter.com/Tree_of_Alpha/status/1580536336250970112'
#authorize twitter login
def oAuth():
    try:
        auth = tweepy.OAuthHandler(twitter_api_key,twitter_secret_key)
        auth.set_access_token(twitter_access_token,twitter_secret_token)
        return auth
    except Exception as e:
        return None

oauth= oAuth()

In [4]:
#set twitter api object and alpaca api object 
twitter_api = tweepy.API(oauth)

alpaca_api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url
)

In [ ]:
#While loop continually pulls most recent tweets from 'News_of_alpha'

"""
While loop continually pulls the most recent tweet from the 'News_of_alpha' twitter account and checks to see
if the the word in the [1] index of the tweet corresponds to 'CPI'. The news of alpha account automates the data 
using a bloomberg terminal, and is a fast way of getting macro data without paying for a terminal. The loop then
grabs the CPI data, and implements a conditional statement deciding if it should trade on the data depending on how
far the actual data is from expectations

"""

while True:
    tweet = twitter_api.user_timeline(
    screen_name = user_id,
    count = 1, 
    include_rts = False,
    exclude_replies = True,
    tweet_mode = 'extended'
)
    #Splits tweet into list of strings 
    for info in tweet:
        tweet = info.full_text
        tweet_string = str(tweet).split()
    #if the index 1 is 'CPI', we know its the right tweet
    if tweet_string[1] == 'CPI:':
        t0= time.time()
        cpi_number = tweet_string[2]
        core_cpi_number = tweet_string[8]
        
        #gather and clean CPI numbers
        cpi = cpi_number.replace('+','')
        cpi = cpi_number.replace('%','')
        cpi = int(float(cpi))

        core_cpi = core_cpi_number.replace('+','')
        core_cpi = core_cpi_number.replace('%','')
        core_cpi = int(float(core_cpi))
        #Conditional statements to determine to long/short market depending on CPI print
        if cpi >= 6.5:
            alpaca_api.submit_order(
            symbol='SPY',
            qty=100,  
            side='sell',
            type='limit',
            limit_price= 375,
            time_in_force='day',
            extended_hours=True,
        )
            #Time how long it takes
            t1 = time.time()
            print(t1-t0)
            break 
        elif cpi <=6.4:
            alpaca_api.submit_order(
            symbol='BTCUSD',
            qty=15,  
            side='buy',
            type='market',
            time_in_force='gtc',
        )
            t1 = time.time() 
            print(t1-t0)
            break
        else:
            print("No trade to be made given current inflation report.")
            t1 = time.time()
            print(t1-t0)
            break